## Module import 

In [17]:
#LGBM
import lightgbm as lgb

#Tensorflow , keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping, Callback
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.utils import to_categorical ,Sequence, get_custom_objects
#from swa.keras import SWA # swa optimizer - https://pypi.org/project/keras-swa/
from tensorflow.keras.models import load_model

#etc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [3]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('tensorflow : %s'%(tf.__version__))
!python --version

Pandas : 1.0.1
Numpy : 1.18.1
tensorflow : 2.2.0
Python 3.7.6


In [4]:
data_df = pd.read_csv('data/201901-202003.csv')
sub_df = pd.read_csv('data/submission.csv')

In [5]:
data_df

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [6]:
sub_df

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,0
1,1,202004,강원,골프장 운영업,0
2,2,202004,강원,과실 및 채소 소매업,0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0
1390,1390,202007,충북,한식 음식점업,0
1391,1391,202007,충북,호텔업,0
1392,1392,202007,충북,화장품 및 방향제 소매업,0


In [ ]:
sub_df['REG_YYMM'].value_counts()

In [8]:
data_df['HOM_CCG_NM'].unique()

array(['강릉시', '속초시', '동해시', '춘천시', '평창군', '성남시 분당구', '안산시 단원구', '용인시 기흥구',
       '용인시 수지구', '강남구', '영월군', '원주시', '정선군', '홍천군', '횡성군', '가평군',
       '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '구리시', '군포시',
       '김포시', '남양주시', '부천시', '성남시 수정구', '성남시 중원구', '수원시 권선구', '수원시 영통구',
       '수원시 장안구', '수원시 팔달구', '시흥시', '안산시 상록구', '안성시', '안양시 동안구',
       '안양시 만안구', '양주시', '양평군', '여주시', '오산시', '용인시 처인구', '의왕시', '의정부시',
       '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '창원시 성산구', '상주시',
       '포항시 남구', '달서구', '대덕구', '서구', '유성구', '강동구', '강북구', '강서구', '관악구',
       '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구',
       '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구',
       '중구', '중랑구', nan, '계양구', '남구', '남동구', '부평구', '연수구', '아산시',
       '천안시 동남구', '천안시 서북구', '괴산군', '제천시', '진천군', '청주시 상당구', '청주시 청원구',
       '청주시 흥덕구', '충주시', '포항시 북구', '북구', '고성군', '삼척시', '양양군', '인제군',
       '태백시', '김해시', '양산시', '창원시 마산합포구', '창원시 진해구', '경산시', '경주시', '구미시',
       '안동시', '울

In [ ]:
data_STDCLSSNM_list = list(data_df['STD_CLSS_NM'].unique())
sub_STDCLSSNM_list = list(sub_df['STD_CLSS_NM'].unique())


In [ ]:
sub_STDCLSSNM_list.sort() == data_STDCLSSNM_list.sort()

In [ ]:
sub_STDCLSSNM_list

In [ ]:
sub_df['CARD_SIDO_NM'].unique()

In [ ]:
sub_df['STD_CLSS_NM'].unique()

In [12]:
df = data_df.copy()
df = df[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
df = df.loc[df['REG_YYMM']==201904]
df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

In [15]:
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.loc[submission['REG_YYMM']==202004]
submission = submission[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submission = submission.merge(df, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission = submission.fillna(0)
AMT = list(submission['AMT'])*2

In [16]:
submission

,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,강원,건강보조식품 소매업,1.065485e+08
1,강원,골프장 운영업,3.964893e+09
2,강원,과실 및 채소 소매업,8.276834e+08
3,강원,관광 민예품 및 선물용품 소매업,1.790561e+07
4,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...
692,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.428269e+09
693,충북,한식 음식점업,2.216287e+10
694,충북,호텔업,4.051093e+07
695,충북,화장품 및 방향제 소매업,6.872543e+08


In [14]:
submission = pd.read_csv('data/submission.csv', index_col=0)
submission['AMT'] = AMT
submission.to_csv('data/submission_myoung.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.065485e+08
1,202004,강원,골프장 운영업,3.964893e+09
2,202004,강원,과실 및 채소 소매업,8.276834e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,1.790561e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [19]:
data_df

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [40]:
data_df[data_df['CARD_SIDO_NM'] == '제주']

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1454855,201901,제주,서귀포시,건강보조식품 소매업,부산,사하구,50s,2,4,4,1900000,4
1454856,201901,제주,서귀포시,건강보조식품 소매업,인천,서구,50s,2,4,3,1300000,2
1454857,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,30s,1,2,3,438000,3
1454858,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,30s,2,2,4,552200,3
1454859,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,40s,1,2,3,908000,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24597421,202003,제주,제주시,휴양콘도 운영업,서울,노원구,50s,1,4,3,79500,3
24597422,202003,제주,제주시,휴양콘도 운영업,제주,제주시,30s,1,2,4,1656000,5
24597423,202003,제주,제주시,휴양콘도 운영업,제주,제주시,40s,1,3,3,304560,3
24597424,202003,제주,제주시,휴양콘도 운영업,제주,제주시,50s,2,4,4,60000,4


In [37]:
cal = data_df['REG_YYMM'] == 202003 & data_df['CARD_SIDO_NM'] == '제주'

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]

In [ ]:
and 'CARD_SIDO_NM' == '제주'